In [ ]:
import paramiko
import subprocess

ip = "192.168.0.21"
user = "root"
pw = "asd123!@"
cmd = ""
SERVER_SMB_DIR = ""
SMB_USERNAME = ""
CLIENT_SMB_DIR = ""
SERVER_IP = "192.168.0.20"
SERVER_SMB_USER = ""
SHARE_NAME = ""

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    (stdin, stdout, stderr) = cli.exec_command(cmd,get_pty = True)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    return out + err

def pp(stdout):
    for i in stdout:
        print(i)
#########################################
#              SMB SERVER               #
#########################################

def smb_install():
    print("smb를 설치합니다")
    cc("dnf update && dnf -y install samba samba-client cifs-utils")
    cc("sudo dnf install -y expect")
    #pp(stdout)

def install_test():
    cc("dpkg -l | grep samba")

def useradd():
    global SMB_USERNAME
    SMB_USERNAME = input("smb user를 생성하기 위한 이름을 입력하세요 (ex. smb)")
    cc(f"useradd -s /sbin/nologin {SMB_USERNAME}")
    PASSWORD = "asd123!@"
    cmd = f'''
    expect -c "
    set timeout 10
    spawn smbpasswd -a {SMB_USERNAME}
    expect \\"New SMB password:\\"
    send \\"{PASSWORD}\\r\\"
    expect \\"Retype new SMB password:\\"
    send \\"{PASSWORD}\\r\\"
    expect eof
    "
    '''
    cc(cmd)

def smb_account_test():
    cmd = "pdbedit --list"
    print(cc(cmd))

def make_directory():
    global SERVER_SMB_DIR
    SERVER_SMB_DIR = input("smb로 공유해줄 서버의 디렉토리를 입력해주세요 ex. /share/smb ") 
    cmd = f"mkdir -p {SERVER_SMB_DIR}"
    cc(cmd)
    print("ls /share 목록입니다 : ")
    print(cc("ls /share"))

def smb_set():
    global SHARE_NAME 
    SHARE_NAME = input("share name을 입력하세요 ex. share")
    cmd = f"""
cat <<EOF >> /etc/samba/smb.conf
[{SHARE_NAME}]
    path = {SERVER_SMB_DIR}
    valid users = {SMB_USERNAME}
    read only = no
    browseable = yes
    writable = yes
    guest ok = no
EOF
"""
    cc(cmd)

def permit():
    cmd = f"""chmod 777 /share
    chown {SMB_USERNAME}:{SMB_USERNAME} {SERVER_SMB_DIR}"""
    cc(cmd)

def service_start():
    cmd = """
systemctl restart smb
systemctl status smb
"""
    print(cc(cmd))
    cmd = """
systemctl restart nmb
systemctl status nmb
"""
    print(cc(cmd))

def smb_server():
    smb_install()       # 설치
    install_test()      # 설치 확인
    useradd()           # smb 계정 생성 및 SMB 계정 등록
    smb_account_test()  # samba 등록 계정 확인
    make_directory()    # smb 계정으로 공유할 디렉터리 생성
    smb_set()           # /etc/samba/smb.conf 파일에 공유폴더 "[share]" 추가
    permit()            # /share : 모두 접근 가능 /share/smb : smb 계정만 접근 및 쓰기 가능
    service_start()     # smbd/nmbd 서비스 활성화

#########################################
#              SMB CLIENT               #
#########################################

def make_directory_client():
    global CLIENT_SMB_DIR
    CLIENT_SMB_DIR = input("smb로 공유받을 클라의 디렉토리를 입력해주세요 ex. /mnt/smb ") 
    cmd = f"mkdir -p {CLIENT_SMB_DIR}"
    cc(cmd)
def mount_temporally_client():
    PASSWORD = "asd123!@"
    global SERVER_IP
    global SERVER_SMB_USER
    global SHARE_NAME 
    SERVER_IP = input("서버로 돌아가 ip -a로 서버의 ip 를 입력해주세요: ")
    SERVER_SMB_USER = input("서버로 돌아가 해당 SMB의 USER를 입력해주세요 ex. smb, smbtest: ")
    SHARE_NAME = input("share name을 입력하세요 ex. share")
    
    # 마운트할 디렉토리 생성
    cc(f"mkdir -p {CLIENT_SMB_DIR}")
    
    # smbclient로 로그인 확인
    print("SMB 서버에 로그인 하겠습니다.")
    
    cmd2 = f"""
cat <<EOF > /root/smb_login.exp
#!/usr/bin/expect -f
set timeout -1
spawn smbclient -U {SERVER_SMB_USER} -L {SERVER_IP}
expect "Password for"
send "{PASSWORD}\\r"
expect eof
EOF
"""
    cc(cmd2)
    cc("chmod +x /root/smb_login.exp")
    print(cc("/usr/bin/expect /root/smb_login.exp"))

    # 실제 mount
    cmd = f' mount -v -t cifs //{SERVER_IP}/{SHARE_NAME} {CLIENT_SMB_DIR} -o username={SERVER_SMB_USER},password="asd123!@",vers=3.0'
    print(cc(cmd))
    #cmd = f"mount -t cifs //{SERVER_IP}/share {CLIENT_SMB_DIR} -o username='{SERVER_SMB_USER}',password='asd123!@',vers=3.0"
    #print(cc(cmd))
    ## 여기 명령어가 안먹습니다.. 직접 입력해주셔야할듯.. ㅠㅠ

def smb_client() :
    smb_install()
    make_directory_client()
    mount_temporally_client()

#실행부
#smb_server()
smb_client()

smb를 설치합니다


smb로 공유받을 클라의 디렉토리를 입력해주세요 ex. /mnt/smb  /mnt/z
서버로 돌아가 ip -a로 서버의 ip 를 입력해주세요:  172.16.16.100
서버로 돌아가 해당 SMB의 USER를 입력해주세요 ex. smb, smbtest:  z


SMB 서버에 로그인 하겠습니다.
